# CIFAR10
* MLP-Mixer， Resnet， MLP, CNN-MLP, VGG-16で比較（予定）

In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2, l1

import models, utils

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

%load_ext autoreload
%autoreload 2

* https://github.com/sayakpaul/MLP-Mixer-CIFAR10/blob/main/MLP_Mixer_Training.ipynb

## 学習

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

### MLP-Mixer

In [173]:
parameters = dict(
    num_blocks = 10,
    patch_size=4, 
    hidden_dim=128, 
    tokens_mlp_dim=64, 
    channels_mlp_dim=128,
    mlp_block_params=dict(
        activation='gelu',
        dropout_rate=0.5,
        dense_params=dict(
            kernel_initializer='he_normal',
        )
    )
)

In [174]:
mixer_model = models.mlp_mixer((32, 32, 3), num_classes=10, **parameters)

In [175]:
mixer_model.compile(Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])

In [176]:
mixer_model.fit(x_train, y_train, validation_split=0.1, epochs=100, batch_size=512)

Epoch 1/100
88/88 [==============================] - 10s 119ms/step - loss: 2.0289 - accuracy: 0.2257 - val_loss: 1.8566 - val_accuracy: 0.2962
Epoch 2/100
88/88 [==============================] - 9s 97ms/step - loss: 1.7540 - accuracy: 0.3364 - val_loss: 1.6688 - val_accuracy: 0.3800
Epoch 3/100
88/88 [==============================] - 9s 97ms/step - loss: 1.5467 - accuracy: 0.4268 - val_loss: 1.4558 - val_accuracy: 0.4750
Epoch 4/100
88/88 [==============================] - 8s 96ms/step - loss: 1.4406 - accuracy: 0.4723 - val_loss: 1.3607 - val_accuracy: 0.4984
Epoch 5/100
88/88 [==============================] - 9s 97ms/step - loss: 1.3796 - accuracy: 0.4970 - val_loss: 1.3127 - val_accuracy: 0.5220
Epoch 6/100
88/88 [==============================] - 9s 97ms/step - loss: 1.3217 - accuracy: 0.5203 - val_loss: 1.2847 - val_accuracy: 0.5412
Epoch 7/100
88/88 [==============================] - 9s 97ms/step - loss: 1.2757 - accuracy: 0.5382 - val_loss: 1.2300 - val_accuracy: 0.5580
Epoc

In [111]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 4s 13ms/step - loss: 1.0963 - accuracy: 0.6991


[1.0963000059127808, 0.6991000175476074]

### Simple-MLP

In [237]:
parameters = dict(
    layers_list=[128, 128, 128],
    activation='relu'
)

In [238]:
mlp_model = models.simple_mlp(input_shape=(32, 32, 3), **parameters)
mlp_model.compile(Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])

In [240]:
mlp_model.fit(x_train, y_train, validation_split=0.1, epochs=10, batch_size=512)

Epoch 1/10
88/88 [==============================] - 1s 8ms/step - loss: 35.3909 - accuracy: 0.1539 - val_loss: 5.6088 - val_accuracy: 0.1762
Epoch 2/10
88/88 [==============================] - 1s 6ms/step - loss: 3.8286 - accuracy: 0.2110 - val_loss: 2.8663 - val_accuracy: 0.2340
Epoch 3/10
88/88 [==============================] - 0s 5ms/step - loss: 2.7686 - accuracy: 0.2323 - val_loss: 3.0729 - val_accuracy: 0.2226
Epoch 4/10
88/88 [==============================] - 0s 6ms/step - loss: 2.4826 - accuracy: 0.2509 - val_loss: 2.2011 - val_accuracy: 0.2676
Epoch 5/10
88/88 [==============================] - 1s 6ms/step - loss: 2.0473 - accuracy: 0.2889 - val_loss: 2.0857 - val_accuracy: 0.2930
Epoch 6/10
88/88 [==============================] - 1s 6ms/step - loss: 1.9978 - accuracy: 0.2979 - val_loss: 2.0893 - val_accuracy: 0.2878
Epoch 7/10
88/88 [==============================] - 0s 6ms/step - loss: 1.9604 - accuracy: 0.3070 - val_loss: 1.9956 - val_accuracy: 0.3074
Epoch 8/10
88/88 [=

In [ ]:
mlp_model.evaluate(x_test, y_test)

In [241]:
mlp_model.summary()

Model: "sequential_190"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_162 (Normaliza (None, 32, 32, 3)         7         
_________________________________________________________________
random_flip_161 (RandomFlip) (None, 32, 32, 3)         0         
_________________________________________________________________
random_rotation_161 (RandomR (None, 32, 32, 3)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_2516 (Dense)           (None, 128)               393344    
_________________________________________________________________
activation_199 (Activation)  (None, 128)               0         
_________________________________________________________________
dense_2517 (Dense)           (None, 128)            

### CNN-MLP

In [242]:
parameters = dict(
    layers_list=[128, 128, 128],
    activation='relu',
)
conv_params=dict(
    filters=128, 
    kernel_size=4
)

In [243]:
cnn_mlp_model = models.cnn_mlp(input_shape=(32, 32, 3), conv_params=conv_params, **parameters)
cnn_mlp_model.compile(Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])

In [244]:
cnn_mlp_model.fit(x_train, y_train, validation_split=0.1, epochs=10, batch_size=512)

Epoch 1/10
88/88 [==============================] - 2s 17ms/step - loss: 104.9430 - accuracy: 0.1766 - val_loss: 5.2729 - val_accuracy: 0.2742
Epoch 2/10
88/88 [==============================] - 1s 15ms/step - loss: 3.4084 - accuracy: 0.2780 - val_loss: 2.4697 - val_accuracy: 0.3170
Epoch 3/10
88/88 [==============================] - 1s 15ms/step - loss: 2.5665 - accuracy: 0.3033 - val_loss: 2.3948 - val_accuracy: 0.3114
Epoch 4/10
88/88 [==============================] - 1s 15ms/step - loss: 2.1525 - accuracy: 0.3292 - val_loss: 2.1010 - val_accuracy: 0.3500
Epoch 5/10
88/88 [==============================] - 1s 15ms/step - loss: 1.9563 - accuracy: 0.3539 - val_loss: 2.0603 - val_accuracy: 0.3534
Epoch 6/10
88/88 [==============================] - 1s 15ms/step - loss: 1.9746 - accuracy: 0.3552 - val_loss: 1.9942 - val_accuracy: 0.3718
Epoch 7/10
88/88 [==============================] - 1s 15ms/step - loss: 1.9006 - accuracy: 0.3679 - val_loss: 2.0281 - val_accuracy: 0.3672
Epoch 8/10


In [165]:
cnn_mlp_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.3027 - accuracy: 0.1000


[2.3027007579803467, 0.10000000149011612]

In [245]:
cnn_mlp_model.summary()

Model: "sequential_191"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_163 (Normaliza (None, 32, 32, 3)         7         
_________________________________________________________________
random_flip_162 (RandomFlip) (None, 32, 32, 3)         0         
_________________________________________________________________
random_rotation_162 (RandomR (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_176 (Conv2D)          (None, 29, 29, 128)       6272      
_________________________________________________________________
flatten_5 (Flatten)          (None, 107648)            0         
_________________________________________________________________
dense_2520 (Dense)           (None, 128)               13779072  
_________________________________________________________________
activation_202 (Activation)  (None, 128)            

## ハイパラ探索


In [3]:
train_params = dict(
    epochs=100,
    batch_size=512,
    learning_rate=0.001
)

In [ ]:
utils.mixer_parameter_search(
    x_train, y_train, x_test, y_test, train_params=train_params, path='./results/epoch200/', start_i=948)

{'epochs': 100, 'batch_size': 512, 'learning_rate': 0.001}
948 {'num_blocks': 24, 'patch_size': 2, 'hidden_dim': 16, 'tokens_mlp_dim': 16, 'channels_mlp_dim': 16, 'mlp_block_params': {'activation': 'gelu', 'dropout_rate': 0, 'batch_norm': True, 'dense_params': {'kernel_initializer': 'he_normal'}}}
313/313 [==============================] - 9s 30ms/step - loss: 1.1629 - accuracy: 0.6062
949 {'num_blocks': 24, 'patch_size': 2, 'hidden_dim': 16, 'tokens_mlp_dim': 16, 'channels_mlp_dim': 16, 'mlp_block_params': {'activation': 'gelu', 'dropout_rate': 0, 'batch_norm': False, 'dense_params': {'kernel_initializer': 'he_normal'}}}
313/313 [==============================] - 9s 28ms/step - loss: 1.1710 - accuracy: 0.5890
950 {'num_blocks': 24, 'patch_size': 2, 'hidden_dim': 16, 'tokens_mlp_dim': 16, 'channels_mlp_dim': 16, 'mlp_block_params': {'activation': 'gelu', 'dropout_rate': 0.5, 'batch_norm': True, 'dense_params': {'kernel_initializer': 'he_normal'}}}
313/313 [=============================